# Fitting a 1D fault slip model to data from the San Andreas fault

Gareth Funning, University of California, Riverside

To demonstrate some very simple features of nonlinear models, I will use one of the very simplest models out there $-$ the 1D model for strike-slip, as popularized by [Savage and Burford (1973)](https://doi.org/10.1029/JB078i005p00832).

## 1. Dependencies 

If you don't have these things installed, you're probably in the wrong conda environment!

In [ ]:
import numpy as np
import pygmt

## 2. The 1D model for strike-slip

Although it is most commonly known as the Savage and Burford model, the 1D model for strike-slip, or more precisely, for a screw dislocation in an elastic half space, was originally derived by Weertman and Weertman (1964). It makes several simplifying assumptions:

- The fault is vertical
- The fault is infinitely long
- The fault extends infinitely deep, beneath a locking depth, $D$
- The fault is locked above the locking depth
- The fault is stably sliding at a constant, uniform rate, $s$, beneath the locking depth

The model describes how the surface deformation rate, $v$, varies as a function of distance, $x$, from the fault. (There are also expressions for strain rate, $e$, which are effectively the spatial derivative of v, but we won't get into that today.) That relationship is analytical, and simple to evaluate:

$$ v = \frac{s}{\pi} \arctan\left(\frac{x}{D}\right)+v_{shift}$$

where $v_{shift}$ is a shift in velocity to account for the difference between zero in the model and any overall regional motion (e.g. due to plate tectonics).

**Looking at the function that describes the model, what are the parameters? Are any of them linear? Are any of them nonlinear?**

The model is often associated with Savage and Burford, two researchers from the USGS, who in 1973 published their application of the model to EDM (Electronic Distance Measurement, not dance music) data from the San Andreas fault system in California. One of the figures in the paper is something of a classic: 

![Savage and Burford figure](https://raw.githubusercontent.com/geniusinaction/GEO244/refs/heads/main/08_nonlinear_inverse_models/savage_burford_1973.png)

The top panel shows the model geometry (left), the theoretical strain rate distribution (center) and expected displacement (or velocity) profile (right). The lower panel shows the same, but for a fault that is stably sliding from the surface (what you would expect if a fault was creeping at its long-term slip rate).


## 3. Evaluating the 1D model

As the model is simple, we can evaluate it easily in Python. We can try to see what happens when we vary the input parameters.

First, we can make a function to evaluate the model to make things simpler. Then we can use PyGMT to plot it.

In [ ]:
# let's define a function for the 1d model:
def slip1d(s, D, v_shift, x):
    v=(s/np.pi)*np.arctan(x/D)+v_shift
    return v

# let's establish some parameter/input values

s=20        # slip rate in mm/yr, positive left-lateral
D=10        # locking depth in km
v_shift=0   # velocity shift in mm/yr

x=np.linspace(-100,100,101)  # an array of distances in km

# and evaluate it!

v=slip1d(s,D,v_shift,x)

# let's plot it to see what we got

region = [np.min(x), np.max(x), -s/2, s/2]

fig = pygmt.Figure()
fig.basemap(region=region,projection='X20c/15c',frame=True)
#fig.plot(data=gps_data,style='c0.1c',fill='black',error_bar='y')
fig.plot(x=x,y=v,pen='2p,navy')
fig.show()


## 4. Solving for a real slip rate and locking depth

Let's try and get hold of some real GNSS data for the San Andreas! Schmalzle et al. (2006) used campaign GNSS survey data from the 1990s and 2000s to constrain its slip rate. The paper is here: https://doi.org/10.1029/2005JB003843

Table 3 of the paper has the relevant data. Can we load it in somehow? And plot it? Note that the fault-parallel velocities are included with their uncertainties $-$ can we load all of those in?

In [ ]:
# a blank code cell for you to contemplate formatting and plotting your data







Notice anything about the sign of the data? Should we do anything about that, and if so, what?

Next, we should think about how we might perform some kind of grid search to solve for the slip rate and locking depth of the San Andreas. The code snippet below is one way of trying to establish some arrays of values over which to evaluate the model. (I would seriously recommend changing these, they were included to give you some idea of syntax, not because they have the correct values!) 

In [ ]:
# set minimum and maximum slip rate bounds, plus a step size
smin=5
smax=50
sstep=5

# make a range of possible slip rate values
slip_rates=np.arange(smin,smax+sstep,sstep)

# set minimum and maximum locking depth bounds, plus a step size
Dmin=5
Dmax=25
Dstep=4

# make a range of possible locking depth values
locking_depths=np.arange(Dmin,Dmax+Dstep,Dstep)

Next, we should try to calculate the fit of a model to the data. One metric we can use is the total squared misfit, also known as the 'Residual Sum of Squares' or *RSS*:

$$ RSS = \sum_{i=1}^N (v_i - v_i')^2 $$

where $N$ is the number of data points, $v_i'$ are the predicted values of the data from the model, and $v_i$ are the observed data values. In vector/matrix form, this becomes:

$$ RSS = (\bf{v - v'})^T (\bf{v-v'}) $$

If you are able to form an inverse covariance matrix, $\bf{E^{-1}}$, based on the uncertainties of the data, you can use it to estimated the Weighted Residual Sum of Squares (*WRSS*) like so:

$$ WRSS = (\bf{v - v'})^T \bf{E^{-1}}(\bf{v-v'}) $$

**Hint**: The `np.diag` function will allow you to make a diagonal matrix out of a numpy array, which might be useful for making $\bf{E^{-1}}$. 

In [1]:
# another empty grid cell for you to contemplate the emptiness of existence,
# or, alternately, evauluating the fit of simple geophysical models, here:









In order to use all of this information to model the slip on the San Andreas fault, we want to set up a couple of loops to evaluate all of the combinations of locking depth and slip rate, and then a misfit (penalty) for each of them. Then, the model with the smallest misfit will be the best-fitting model!

The following code snippet will loop through all of your locking depths, as a starting point:

In [ ]:
# loop through your locking depths to show how it is done
for D in locking_depths:
    print(D)  # mind you, you probably want to evaluate the model here, and then the misfit, not whatever this is

In [ ]:
# a code cell to wonder about plotting the penalty function (i.e. the misfit)








## References


Savage, J. C., and R. O. Burford (1973), Geodetic determination of relative plate motion in central California, J. Geophys. Res., 78(5), 832–845, [doi:10.1029/JB078i005p00832](https://doi.org/10.1029/JB078i005p00832). 

Schmalzle, G., T. Dixon, R. Malservisi, and R. Govers (2006), Strain accumulation across the Carrizo segment of the San Andreas Fault, California: Impact of laterally varying crustal properties, J. Geophys. Res., 111, B05403, [doi:10.1029/2005JB003843]( https://doi.org/10.1029/2005JB003843).  
